In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import random
from shutil import copyfile

# Créer les dossiers train et test
try:
    os.mkdir('train')
    os.mkdir('test')
except OSError:
    pass

# Définir les noms des classes et les chemins d'accès aux dossiers
class_names = ['Droun', 'noise']
data_paths = ['/content/drive/MyDrive/Colab Notebooks/data/droun', '/content/drive/MyDrive/Colab Notebooks/data/noise']

# Définir la proportion de données à utiliser pour l'ensemble de test
test_size = 0.2

# Pour chaque classe, diviser les données en ensembles de train et de test
for i, class_name in enumerate(class_names):
    files = os.listdir(data_paths[i])
    random.shuffle(files)
    n_test = int(len(files) * test_size)
    test_files = files[:n_test]
    train_files = files[n_test:]

    # Copier les fichiers dans les dossiers train et test
    for file_name in train_files:
        src_path = os.path.join(data_paths[i], file_name)
        dst_path = os.path.join('train', class_name, file_name)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        copyfile(src_path, dst_path)

    for file_name in test_files:
        src_path = os.path.join(data_paths[i], file_name)
        dst_path = os.path.join('test', class_name, file_name)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        copyfile(src_path, dst_path)


In [ ]:
!ls '/content'

In [ ]:
DATA_DIR= '/content/drive/MyDrive/data/DATA_sound'
print(DATA_DIR)

In [ ]:
!pip install YOLOv5

In [ ]:
#!pip install ultralytics 8.0.112
!pip install ultralytics==8.0.73

In [ ]:
import ultralytics
print(ultralytics.__version__)

In [ ]:
import os
from ultralytics import YOLO

# create an instance of the YOLOv5 model

# Load a model
model = YOLO("yolov8n-cls.pt")  # load a pretained model

# train the model
results = model.train(data=DATA_DIR, epochs=10, imgsz=64)  # train the mode

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import seaborn as sns
from google.colab import files
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [ ]:
results_path = '/content/runs/classify/train/results.csv'

results = pd.read_csv(results_path)

plt.figure()
plt.plot(results['                  epoch'], results['             train/loss'], label='train loss')
plt.plot(results['                  epoch'], results['               val/loss'], label='val loss', c='red')
plt.grid()
plt.title('Loss vs epochs')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()

plt.savefig( " Loss_vs_epochs.png")

plt.figure()
plt.plot(results['                  epoch'], results['  metrics/accuracy_top1'] * 100)
plt.grid()
plt.title('Validation accuracy vs epochs')
plt.ylabel('accuracy (%)')
plt.xlabel('epochs')
plt.savefig( " Validation_accuracy_vs_epochs.png")
#files.download ( " Loss_vs_epochs.png" )
#files.download ( " Validation_accuracy_vs_epochs.png" )
plt.show()

In [ ]:
from ultralytics import YOLO
import numpy as np
model = YOLO('/content/runs/classify/train/weights/last.pt')  # load a custom model

sound = '/content/drive/MyDrive/data/DATA_sound/test/droun/1.wav'
y, sr = librosa.load(sound)
#print(sr)
mel_spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512  , n_mels=128)
mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
librosa.display.specshow(mel_spect, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');
plt.savefig( " Mel Spectrogram.png")




results = model('Mel Spectrogram.png')  # predict
#print(results)

names_dict = results[0].names
#print(names_dict)

probs = results[0].probs.tolist()


print(probs)
class_binar = np.argmax(probs)
print(class_binar)
class_name = names_dict[np.argmax(probs)]
print(class_name)

In [ ]:
!pip install scikit-learn

In [ ]:
path='/content/drive/MyDrive/data/DATA_sound/test'

y_droun=[]
y_noise=[]

y_test = [y_droun,y_noise]


person_folders=os.listdir(path) #list the content of image_crop folder
print(person_folders)

In [ ]:
j=0
for i in person_folders: # a loop to store label names inside the person_rep dictionnary
  print(i)
  image_names=os.listdir(path+'/'+str(i)+'/') #list the content (images) of each subject folder of the training set
  print(np.shape(image_names))
  for image_name in image_names: # a loop to read all testing images
   #print(image_name)
   #img=load_img(path+'/'+str(i)+'/'+image_name,target_size=(64,64))
   #print(img)

   y_test[j].append(j) #append the label to the y_train array
   #print(j)
  j=j+1

In [ ]:
print(y_droun)
print('y_droun shape : {}'.format(np.shape(y_droun)))
print(y_noise)
print('y_noise shape : {}'.format(np.shape(y_noise)))

In [ ]:
y_test = np.concatenate((y_droun,y_noise), axis = 0)
print(y_test)
print('y_test shape : {}'.format(np.shape(y_test)))

In [ ]:
from ultralytics import YOLO
import numpy as np
model = YOLO('/content/runs/classify/train/weights/last.pt')  # load a custom model

In [ ]:
prediction_test = []
prediction_class = []
j=0
for i in person_folders: # a loop to store label names inside the person_rep dictionnary
  print(i)
  image_names=os.listdir(path+'/'+str(i)+'/') #list the content (images) of each subject folder of the training set
  print(np.shape(image_names))
  for image_name in image_names: # a loop to read all testing images
   #print(image_name)

   img=load_img(path+'/'+str(i)+'/'+image_name)
   #print(img)
   results = model(img)  # predict on an image
   #print(results)

   names_dict = results[0].names
   #print(names_dict)

   probs = results[0].probs.tolist()
   class_binar = np.argmax(probs)
   print(class_binar)
   class_name = names_dict[np.argmax(probs)]
   print(class_name)


   prediction_test.append(class_binar) #append the label to the y_train array
   prediction_class.append(class_name)
   #print(j)
  j=j+1

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,prediction_test)

# Calculate AUC score
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig( "Receiver operating characteristic.png")
#files.download( " Receiver operating characteristic.png" )
plt.show()

In [ ]:
cm = confusion_matrix(y_test, prediction_test)
print(cm)
fig = plt.figure(figsize = (4,5))
ax = fig.add_subplot(1,1,1)
ax = sns.heatmap(cm, annot=True,cmap='Purples',fmt=".0f" )

#ax.set_title('Seaborn Confusion Matrix with labels\n\n',fontsize = 10);
ax.set_xlabel('\nPredicted Values',fontsize = 10)
ax.set_ylabel('Actual Values ',fontsize = 10);

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Mask','Real'])
ax.yaxis.set_ticklabels(['Mask','Real'])

plt.savefig( " Confusion Matrix.png")

#files.download( " Confusion Matrix.png" )

## Display the visualization of the Confusion Matrix.
plt.show()